<a href="https://colab.research.google.com/github/grbarbosa3/data_science/blob/master/PROD6_modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split
from sklearn import metrics
import numpy as np
import seaborn as sns


In [ ]:
df_c = pd.read_csv("/content/dCliente.csv")
df_t = pd.read_csv("/content/transac.csv",sep=';',error_bad_lines=False, names=('COD_ASSOCIADO', 'EMPRESA', 'ESTABELECIMENTO', 'DATA_MOVIMENTO', 'HORA', 'VALOR', 'AUTORIZACAO', 'COD_ESTABELECIMENTO', 'LANCAMENTO', 'IDENTIFICACAO','PARCELAS'))

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


##Tabela de Transações

In [ ]:
df_t.head()

,COD_ASSOCIADO,EMPRESA,ESTABELECIMENTO,DATA_MOVIMENTO,HORA,VALOR,AUTORIZACAO,COD_ESTABELECIMENTO,LANCAMENTO,IDENTIFICACAO,PARCELAS
0,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-03-10 00:00:00.000,15:29,188.00,55691728,10385.0,COMPRA NO PARCELADO,PAG*RaildaAbrantes,5
1,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-03-10 00:00:00.000,15:29,188.00,55691728,10385.0,COMPRA NO PARCELADO,PAG*RaildaAbrantes,5
2,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-03-10 00:00:00.000,15:29,188.00,55691728,10385.0,COMPRA NO PARCELADO,PAG*RaildaAbrantes,5
3,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-03-10 00:00:00.000,15:29,188.00,55691728,10385.0,COMPRA NO PARCELADO,PAG*RaildaAbrantes,5
4,4185782,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-03-18 00:00:00.000,09:22,133.74,146699363,10385.0,COMPRA NO PARCELADO,AUTO SERVICO INTERNACI,2


In [ ]:
# Tirando transções duplicadas

df_t2 = df_t.drop_duplicates(subset='AUTORIZACAO')

In [ ]:
# Mudando formato de data para coluna DATA_MOVIMENTO

df_t2['DATA_MOVIMENTO'] = pd.to_datetime(df_t2['DATA_MOVIMENTO'], format="%Y-%m-%d")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## Tabela de Cliente

In [ ]:
df_c.head()

,COD_ASSOCIADO,COD_EMPRESA,STATUS,LIMITE,DATA_PRIMEIRA_COMPRA,DATA_ULTIMA_COMPRA,CONSUMO,VIA_CARTAO,ADESAO,TIPO_PESSOA,SEXO,ESTADO_CIVIL,NASCIMENTO,RENDA,FECHAMENTO,RENDA_COMPROVADA,DIA_VENCTO,ESCOLARIDADE,OCUPACAO,RISCO_LOJISTA,NOME,COD_ESTADO,ULTIMA_ATUALIZACAO
0,1687074,206,L,"1000,00",2014-10-27 00:00:00.000,2021-04-24 00:00:00.000,"96,62",3,2020-10-15 00:00:00.000,F,F,SOLTEIRO,1994-05-05 00:00:00.000,"1100,00",27,S,11,COLEGIAL,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO PAULO,SP,2014-10-27 15:09:00.960
1,1687201,206,L,"150,00",2014-10-27 00:00:00.000,2014-10-27 00:00:00.000,"0,00",3,2020-10-15 00:00:00.000,F,M,SOLTEIRO,1960-11-29 00:00:00.000,"3500,00",28,S,12,COLEGIAL,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO PAULO,SP,2014-10-27 16:41:22.153
2,1695640,206,L,"1149,90",2014-11-08 00:00:00.000,2021-04-12 00:00:00.000,"1112,33",3,2020-10-15 00:00:00.000,F,M,SOLTEIRO,1978-05-04 00:00:00.000,"3000,00",8,S,23,COLEGIAL,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO PAULO,SP,2021-02-02 21:00:10.797
3,1695909,206,Q,"1100,00",2014-11-08 00:00:00.000,2021-04-14 00:00:00.000,"3,99",3,2020-10-15 00:00:00.000,F,F,CASADO,1987-09-02 00:00:00.000,"3000,00",28,S,12,COLEGIAL,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO PAULO,SP,2014-11-08 09:47:20.887
4,1698055,206,L,"700,00",2014-11-11 00:00:00.000,2021-04-20 00:00:00.000,"514,05",5,2020-10-15 00:00:00.000,F,F,DIVORCIADO,1970-10-16 00:00:00.000,"1096,00",27,S,11,COLEGIAL,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO PAULO,SP,2014-11-11 15:49:50.230


In [ ]:
 # Trocando virgula por ponto nos valores numéricos
df_c['LIMITE'] = df_c['LIMITE'].apply(lambda x: x.replace(',','.'))

In [ ]:
# Trocando valores para float
df_c['LIMITE'] = df_c['LIMITE'].astype(float, errors = 'raise') 


In [ ]:
# df_outliers = df_c.loc[df_c['LIMITE'] > 5000]
# df_outliers['LIMITE'].value_counts().sum()
# df_outliers['LIMITE'].value_counts()
# df_outliers2 = df.loc[df['LIMITE'] > 50000]

In [ ]:
df_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62018 entries, 0 to 62017
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   COD_ASSOCIADO         62018 non-null  int64  
 1   COD_EMPRESA           62018 non-null  int64  
 2   STATUS                62018 non-null  object 
 3   LIMITE                62018 non-null  float64
 4   DATA_PRIMEIRA_COMPRA  61122 non-null  object 
 5   DATA_ULTIMA_COMPRA    61122 non-null  object 
 6   CONSUMO               62018 non-null  object 
 7   VIA_CARTAO            62018 non-null  int64  
 8   ADESAO                62018 non-null  object 
 9   TIPO_PESSOA           62018 non-null  object 
 10  SEXO                  62018 non-null  object 
 11  ESTADO_CIVIL          52342 non-null  object 
 12  NASCIMENTO            62018 non-null  object 
 13  RENDA                 62018 non-null  object 
 14  FECHAMENTO            62018 non-null  int64  
 15  RENDA_COMPROVADA   

In [ ]:
#Trocando formato de valores de data
df_c['DATA_PRIMEIRA_COMPRA'] = pd.to_datetime(df_c['DATA_PRIMEIRA_COMPRA'], format="%Y-%m-%d")
df_c['DATA_ULTIMA_COMPRA'] = pd.to_datetime(df_c['DATA_ULTIMA_COMPRA'], format="%Y-%m-%d")
df_c['ADESAO'] = pd.to_datetime(df_c['ADESAO'], format="%Y-%m-%d")
df_c['NASCIMENTO'] = pd.to_datetime(df_c['NASCIMENTO'], format="%Y-%m-%d")
df_c['ULTIMA_ATUALIZACAO'] = pd.to_datetime(df_c['ULTIMA_ATUALIZACAO'], format="%Y-%m-%d")

In [ ]:
#criando associaçoes para dar MERGE nas colunas
left = df_t2
right = df_c

In [ ]:
#Novo dataframe com o merge
df = pd.merge(left, right, on="COD_ASSOCIADO")

## Novo DATAFRAME Uniao de Cliente com Transações

In [ ]:
df.head()

,COD_ASSOCIADO,EMPRESA,ESTABELECIMENTO,DATA_MOVIMENTO,HORA,VALOR,AUTORIZACAO,COD_ESTABELECIMENTO,LANCAMENTO,IDENTIFICACAO,PARCELAS,COD_EMPRESA,STATUS,LIMITE,DATA_PRIMEIRA_COMPRA,DATA_ULTIMA_COMPRA,CONSUMO,VIA_CARTAO,ADESAO,TIPO_PESSOA,SEXO,ESTADO_CIVIL,NASCIMENTO,RENDA,FECHAMENTO,RENDA_COMPROVADA,DIA_VENCTO,ESCOLARIDADE,OCUPACAO,RISCO_LOJISTA,NOME,COD_ESTADO,ULTIMA_ATUALIZACAO
0,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-03-10,15:29,188.00,55691728,10385.0,COMPRA NO PARCELADO,PAG*RaildaAbrantes,5,318,L,4000.0,2020-12-18,2021-04-28,"3972,37",1,2020-11-01,F,F,OUTROS,1965-12-22,"1000,00",8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177
1,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-03-10,15:29,188.00,55691728,10385.0,COMPRA NO PARCELADO,PAG*RaildaAbrantes,5,318,L,4000.0,2020-12-18,2021-04-28,"3972,37",1,2020-11-01,F,F,OUTROS,1965-12-22,"1000,00",8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.180
2,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2020-12-19,10:07,123.77,1927289611,10385.0,COMPRA NO PARCELADO,MOVEIS LINHARES,10,318,L,4000.0,2020-12-18,2021-04-28,"3972,37",1,2020-11-01,F,F,OUTROS,1965-12-22,"1000,00",8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177
3,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2020-12-19,10:07,123.77,1927289611,10385.0,COMPRA NO PARCELADO,MOVEIS LINHARES,10,318,L,4000.0,2020-12-18,2021-04-28,"3972,37",1,2020-11-01,F,F,OUTROS,1965-12-22,"1000,00",8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.180
4,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-04-01,11:02,50.30,2023192187,10385.0,COMPRA NO PARCELADO,EXTRABOM SUPERMERCADOS,2,318,L,4000.0,2020-12-18,2021-04-28,"3972,37",1,2020-11-01,F,F,OUTROS,1965-12-22,"1000,00",8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177


In [ ]:
# Trocando pontos por virgulas em valores numéricos

df['CONSUMO'] = df['CONSUMO'].apply(lambda x: x.replace(',','.'))
df['RENDA'] = df['RENDA'].apply(lambda x: x.replace(',','.'))
df['CONSUMO'] = df['CONSUMO'].astype(float, errors = 'raise') 
df['RENDA'] = df['RENDA'].astype(float, errors = 'raise') 

In [ ]:
# Dropando novamente baseado na coluna AUTORIZACAO e gerando um novo Dataframe

df2 = df.drop_duplicates(subset='AUTORIZACAO')

In [ ]:
df2.shape

(1400133, 33)

In [ ]:
df2.head()

,COD_ASSOCIADO,EMPRESA,ESTABELECIMENTO,DATA_MOVIMENTO,HORA,VALOR,AUTORIZACAO,COD_ESTABELECIMENTO,LANCAMENTO,IDENTIFICACAO,PARCELAS,COD_EMPRESA,STATUS,LIMITE,DATA_PRIMEIRA_COMPRA,DATA_ULTIMA_COMPRA,CONSUMO,VIA_CARTAO,ADESAO,TIPO_PESSOA,SEXO,ESTADO_CIVIL,NASCIMENTO,RENDA,FECHAMENTO,RENDA_COMPROVADA,DIA_VENCTO,ESCOLARIDADE,OCUPACAO,RISCO_LOJISTA,NOME,COD_ESTADO,ULTIMA_ATUALIZACAO
0,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-03-10,15:29,188.00,55691728,10385.0,COMPRA NO PARCELADO,PAG*RaildaAbrantes,5,318,L,4000.0,2020-12-18,2021-04-28,3972.37,1,2020-11-01,F,F,OUTROS,1965-12-22,1000.0,8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177
2,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2020-12-19,10:07,123.77,1927289611,10385.0,COMPRA NO PARCELADO,MOVEIS LINHARES,10,318,L,4000.0,2020-12-18,2021-04-28,3972.37,1,2020-11-01,F,F,OUTROS,1965-12-22,1000.0,8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177
4,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-04-01,11:02,50.30,2023192187,10385.0,COMPRA NO PARCELADO,EXTRABOM SUPERMERCADOS,2,318,L,4000.0,2020-12-18,2021-04-28,3972.37,1,2020-11-01,F,F,OUTROS,1965-12-22,1000.0,8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177
6,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-04-28,17:34,51.41,4166399795,10385.0,COMPRA NO ROTATIVO,SUPERMERCADO YPE,1,318,L,4000.0,2020-12-18,2021-04-28,3972.37,1,2020-11-01,F,F,OUTROS,1965-12-22,1000.0,8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177
8,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-01-29,15:46,179.80,5995589954,10385.0,COMPRA NO PARCELADO,CASAS BAHIA,5,318,L,4000.0,2020-12-18,2021-04-28,3972.37,1,2020-11-01,F,F,OUTROS,1965-12-22,1000.0,8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177


In [ ]:
# Tirando colunas sem utilidade para o modelo 

df_clean = df2.drop(columns=['AUTORIZACAO', 'ESTABELECIMENTO', 'IDENTIFICACAO', 'EMPRESA', 'ESCOLARIDADE', 'OCUPACAO', 'RISCO_LOJISTA'])
df_clean = df2.drop(columns=['TIPO_PESSOA'])

In [ ]:
df_clean.head()

,COD_ASSOCIADO,EMPRESA,ESTABELECIMENTO,DATA_MOVIMENTO,HORA,VALOR,AUTORIZACAO,COD_ESTABELECIMENTO,LANCAMENTO,IDENTIFICACAO,PARCELAS,COD_EMPRESA,STATUS,LIMITE,DATA_PRIMEIRA_COMPRA,DATA_ULTIMA_COMPRA,CONSUMO,VIA_CARTAO,ADESAO,SEXO,ESTADO_CIVIL,NASCIMENTO,RENDA,FECHAMENTO,RENDA_COMPROVADA,DIA_VENCTO,ESCOLARIDADE,OCUPACAO,RISCO_LOJISTA,NOME,COD_ESTADO,ULTIMA_ATUALIZACAO
0,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-03-10,15:29,188.00,55691728,10385.0,COMPRA NO PARCELADO,PAG*RaildaAbrantes,5,318,L,4000.0,2020-12-18,2021-04-28,3972.37,1,2020-11-01,F,OUTROS,1965-12-22,1000.0,8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177
2,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2020-12-19,10:07,123.77,1927289611,10385.0,COMPRA NO PARCELADO,MOVEIS LINHARES,10,318,L,4000.0,2020-12-18,2021-04-28,3972.37,1,2020-11-01,F,OUTROS,1965-12-22,1000.0,8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177
4,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-04-01,11:02,50.30,2023192187,10385.0,COMPRA NO PARCELADO,EXTRABOM SUPERMERCADOS,2,318,L,4000.0,2020-12-18,2021-04-28,3972.37,1,2020-11-01,F,OUTROS,1965-12-22,1000.0,8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177
6,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-04-28,17:34,51.41,4166399795,10385.0,COMPRA NO ROTATIVO,SUPERMERCADO YPE,1,318,L,4000.0,2020-12-18,2021-04-28,3972.37,1,2020-11-01,F,OUTROS,1965-12-22,1000.0,8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177
8,4230899,DMCARD MASTERCARD,DMCARD MASTERCARD VITORIA,2021-01-29,15:46,179.80,5995589954,10385.0,COMPRA NO PARCELADO,CASAS BAHIA,5,318,L,4000.0,2020-12-18,2021-04-28,3972.37,1,2020-11-01,F,OUTROS,1965-12-22,1000.0,8,S,23,NENHUMA,FUNCIONÁRIO REGISTRADO/ASSALARIADO,N,SAO MATEUS,ES,2020-11-01 05:02:51.177


In [ ]:
# Eliminando valores inuteis relativos ao LANCAMENTO

LISTA_LANCAMENTO = ['COMPRA NO ROTATIVO','COMPRA NO PARCELADO','ESTORNO COMPRA ROTATIVO','ESTORNO COMPRA NO PARCELADO','IOF INTERNACIONAL']

df_clean['LANCAMENTO'] = df_clean['LANCAMENTO'].apply(lambda x: x if x in LISTA_LANCAMENTO 
                                          else 'OUTROS')

In [ ]:
# Fatorizando colunas com variáveis categoricas ("Binarizando")

lancamento_encoded, lancamento_cat = df_clean['LANCAMENTO'].factorize()
status_encoded, status_cat = df_clean['STATUS'].factorize()
sexo_encoded, sexo_cat = df_clean['SEXO'].factorize()
estado_civil_encoded, estado_civil_cat = df_clean['ESTADO_CIVIL'].factorize()
cidade_encoded, cidade_cat = df_clean['NOME'].factorize()
estado_encoded, estado_cat = df_clean['COD_ESTADO'].factorize()
renda_encoded, renda_cat = df_clean['RENDA_COMPROVADA'].factorize()

In [ ]:
# Substituindo valores fatorizados nas colunas

df_clean['LANCAMENTO'] = lancamento_encoded
df_clean['STATUS'] = status_encoded
df_clean['SEXO'] = sexo_encoded
df_clean['ESTADO_CIVIL'] = estado_civil_encoded
df_clean['NOME'] = cidade_encoded
df_clean['COD_ESTADO'] = estado_encoded
df_clean['RENDA_COMPROVADA'] = renda_encoded

## Matriz de correlação e Arvore de Decisão

In [ ]:
ax = sns.heatmap(
    corrMatrix, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

NameError: ignored

In [ ]:
feature_cols = ['VALOR','LANCAMENTO','PARCELAS','LIMITE','CONSUMO','VIA_CARTAO','SEXO','ESTADO_CIVIL','RENDA','FECHAMENTO','RENDA_COMPROVADA','DIA_VENCTO','NOME','COD_ESTADO']
X = df_clean[feature_cols]
y = df_clean['STATUS']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)

In [ ]:
clf = DecisionTreeClassifier()
clf = clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)


In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

In [ ]:
pip install azureml-core

In [ ]:
pip install azureml-train

In [ ]:
pip install azureml-sdk


In [ ]:
pip install azureml-interpret

In [ ]:
import logging
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.train.automl import AutoMLConfig
from azureml.interpret import ExplanationClient

In [ ]:
sudo pip install autosklearn